In [ ]:
import pandas as pd

In [ ]:
base_directory = 'khaleesi/data/'

http_features_dir = base_directory + 'http-chains-features.csv'
js_features_dir = base_directory + 'js-chains-features.csv'

all_encoded_features = base_directory + 'all-features-encoded.csv'

In [ ]:
http_dataset = pd.read_csv(http_features_dir)
js_dataset = pd.read_csv(js_features_dir)

In [ ]:
http_dataset['http/js'] = [0] * len(http_dataset)
js_dataset['http/js'] = [1] * len(js_dataset)

dataset = pd.concat([http_dataset, js_dataset], sort=False)

In [ ]:
encoded_dataset = pd.DataFrame()

column_name_count = set()

unchanged_columns = ['identifier', 'length_of_url', 'has_subdomains', 'subdomain_of_top_level_domain_check',\
                     'url_has_uuid', 'url_has_dimensions', 'third_party_domain', \
                     'num_non_alphanumeric_chars_in_query_string', 'top_domain_in_query_string',\
                     'num_request_cookies', 'semi_colons_in_url', 'query_string_length', 'keyword_in_url_re',\
                     'keyword_in_url', 'length_of_chain', 'num_unique_domains', 'num_request_headers', 'http/js']

for element in unchanged_columns:
    encoded_dataset[element] = dataset[element]

In [ ]:
to_encode = ['request_method', 'response_status', 'etag_in_header', 'p3p_in_header', 'resource_type',\
             'response_sets_cookie', 'response_type', 'response_subtype', 'redirect_to_new_domain']

for element in to_encode:
    encoded_element = pd.get_dummies(dataset[element])
    for key in encoded_element:
        if key in column_name_count:
            encoded_dataset[key + '_' + element] = encoded_element[key]
        else:
            encoded_dataset[key] = encoded_element[key]
            column_name_count.add(key)

In [ ]:
def average_column(column):
    values = []
    
    for num in dataset[column]:
        if num != '?':
            values.append(int(num))
    
    return sum(values) / len(values)

In [ ]:
to_replace = ['content_length', 'num_response_headers']

for element in to_replace:
    encoded_dataset[element] = dataset[element].replace('?', average_column(element))

In [ ]:
encoded_dataset['target'] = dataset['target']

In [ ]:
encoded_dataset.to_csv(all_encoded_features, index=False)